In [1]:
import os
from dotenv import load_dotenv
import pandas as pd

In [2]:
load_dotenv()
gemini_api_key =os.getenv("GEMINI_API_KEY")
print('loaded succesfully')

loaded succesfully


In [3]:
df= pd.read_csv("twcs.csv")
df.tail()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
2811769,2987947,sprintcare,False,Wed Nov 22 08:43:51 +0000 2017,"@823869 Hey, we'd be happy to look into this f...",NaN,2987948.0
2811770,2987948,823869,True,Wed Nov 22 08:35:16 +0000 2017,@115714 wtf!? I’ve been having really shitty s...,2987947,NaN
2811771,2812240,121673,True,Thu Nov 23 04:13:07 +0000 2017,@143549 @sprintcare You have to go to https://...,NaN,2812239.0
2811772,2987949,AldiUK,False,Wed Nov 22 08:31:24 +0000 2017,"@823870 Sounds delicious, Sarah! 😋 https://t.c...",NaN,2987950.0
2811773,2987950,823870,True,Tue Nov 21 22:01:04 +0000 2017,@AldiUK warm sloe gin mince pies with ice cre...,"2987951,2987949",NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2811774 entries, 0 to 2811773
Data columns (total 7 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tweet_id                 int64  
 1   author_id                object 
 2   inbound                  bool   
 3   created_at               object 
 4   text                     object 
 5   response_tweet_id        object 
 6   in_response_to_tweet_id  float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 131.4+ MB


In [5]:
df.isnull().sum()

tweet_id                         0
author_id                        0
inbound                          0
created_at                       0
text                             0
response_tweet_id          1040629
in_response_to_tweet_id     794335
dtype: int64

In [6]:
df.shape

(2811774, 7)

In [7]:
questions_df =df[df["inbound"] ==True].copy()

In [8]:
questions_df.shape

(1537843, 7)

In [9]:
questions_df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
6,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN
8,12,115713,True,Tue Oct 31 22:04:47 +0000 2017,@sprintcare You gonna magically change your co...,"11,13,14",15.0


In [10]:
answers_df =df[df["inbound"] == False].copy()

In [11]:
answers_df.shape

(1273931, 7)

In [12]:
answers_df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
5,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0
7,11,sprintcare,False,Tue Oct 31 22:10:35 +0000 2017,@115713 This is saddening to hear. Please shoo...,NaN,12.0
9,15,sprintcare,False,Tue Oct 31 20:03:31 +0000 2017,@115713 We understand your concerns and we'd l...,12,16.0


In [13]:
merge_df =pd.merge(
    answers_df,
    questions_df,
    left_on ='in_response_to_tweet_id',
    right_on ='tweet_id'
)


In [14]:
knowledge_base_df =merge_df[['text_y','text_x']].copy()

In [15]:
knowledge_base_df.rename(columns={'text_y' :'question','text_x' :'answer'}, inplace=True)

In [16]:
display(knowledge_base_df.head())

,question,answer
0,@sprintcare I have sent several private messag...,@115712 I understand. I would like to assist y...
1,@sprintcare I did.,@115712 Please send us a Private Message so th...
2,@sprintcare is the worst customer service,@115712 Can you please send us a private messa...
3,@sprintcare You gonna magically change your co...,@115713 This is saddening to hear. Please shoo...
4,@sprintcare Since I signed up with you....Sinc...,@115713 We understand your concerns and we'd l...


In [17]:
knowledge_base_df.shape

(1261888, 2)

In [23]:
knowledge_base_sample_df =knowledge_base_df.sample(n=200000, random_state=42).copy()


In [27]:
import re
def clean_tweet_text(text):
    text = str(text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'#\w+', '', text)
    return text.strip().lower()
knowledge_base_sample_df['cleaned_question'] = knowledge_base_sample_df['question'].apply(clean_tweet_text)

knowledge_base_sample_df

,question,answer,cleaned_question
976336,@ATVIAssist When is new update,@676038 We're a tech support team here Jonatha...,when is new update
232797,@AppleSupport Some time ago I had an appleid (...,@259371 We'll do everything we can to get you ...,some time ago i had an appleid (which was an e...
454689,Vor über 2 Wochen an den Amazon Kundendienst g...,@385776 Hast du dich denn über unser Kontaktfo...,vor über 2 wochen an den amazon kundendienst g...
753991,"@115817. If your customer service was helpful,...",@551746 I'm sorry for the frustrations. Please...,". if your customer service was helpful, i woul..."
1255132,@115955 #Spectrum the worst TV service ever. ...,@613060 Hello Paul. My apologies for any frust...,the worst tv service ever. just fix the on de...
...,...,...,...
730779,@hulu_support How many people can watch my hul...,"@538086 Hi, Tim! On-demand plans come with 1 s...",how many people can watch my hulu at the same ...
1065957,I finally finished up a nice 4 hour stream and...,"@722601 Thank you for reaching out, if you can...",i finally finished up a nice 4 hour stream and...
579598,@Tesco Call of duty ww2 btw,"@452131 Hi Rob, Streatham Extra will be the cl...",call of duty ww2 btw
708009,@VirginTrains any idea how delayed the 21.07 t...,"@524994 Hi Maggie, we are currently awaiting f...",any idea how delayed the 21.07 to liverpool is?


In [31]:
from langdetect import detect,LangDetectException
from tqdm import tqdm
def is_english(text):
    try:
        if isinstance(text, str) and len(text.strip()) > 0:
            return detect(text) == 'en'
    except LangDetectException:
        pass
    return False
#Create a boolean mask for English questions
# We use tqdm to show progress as this step can be slow
tqdm.pandas(desc="Language Detection")
english_mask = knowledge_base_sample_df['cleaned_question'].progress_apply(is_english)

# Apply the mask to create a new, English-only DataFrame
english_kb_df = knowledge_base_sample_df[english_mask].copy()

english_kb_df.head()

Language Detection: 100%|█████████████████████████████████████████████████████| 200000/200000 [08:06<00:00, 411.09it/s]


,question,answer,cleaned_question
976336,@ATVIAssist When is new update,@676038 We're a tech support team here Jonatha...,when is new update
232797,@AppleSupport Some time ago I had an appleid (...,@259371 We'll do everything we can to get you ...,some time ago i had an appleid (which was an e...
753991,"@115817. If your customer service was helpful,...",@551746 I'm sorry for the frustrations. Please...,". if your customer service was helpful, i woul..."
1255132,@115955 #Spectrum the worst TV service ever. ...,@613060 Hello Paul. My apologies for any frust...,the worst tv service ever. just fix the on de...
839108,".@116062 yesterday in Lake Bluff, IL store ALL...",@600105 We love to hear about your great exper...,". yesterday in lake bluff, il store all associ..."


In [35]:
if len(english_kb_df) >100000:
    knowledge_base_100K_sample_df =english_kb_df.sample(n=100000, random_state=42).copy()
else:                   #if fewer than 100k smaples
    knowledge_base_100K_sample_df =english_kb_df
print(f'final data size is {len(knowledge_base_100K_sample_df)}')
#loading model
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

question_to_embed =knowledge_base_100K_sample_df['cleaned_question'].tolist()

embedding =model.encode(question_to_embed,show_progress_bar=True)

print('\n embedding complete')
embedding.shape

final data size is 100000


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\balap\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\balap\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/3125 [00:00<?, ?it/s]

C:\Users\balap\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



 embedding complete


(100000, 384)

In [37]:
import faiss
import numpy as np

# This gets the dimension of our vectors (e.g., 384)
vector_dimension = embedding.shape[1]

# This creates a standard, accurate FAISS index
index = faiss.IndexFlatL2(vector_dimension)

# This adds our 100,000 vectors to the index
index.add(embedding)

# This saves our finished index to a permanent file
faiss.write_index(index, 'knowledge_base.index')

print(f"Index created and saved. It contains {index.ntotal} vectors.")

Index created and saved. It contains 100000 vectors.


In [41]:
knowledge_base_100K_sample_df.columns

Index(['question', 'answer', 'cleaned_question'], dtype='object')

In [43]:
knowledge_base_100K_sample_df.to_csv('knowledge_base_100k_csv_sample.csv',index=False)
print('success')

success


# Librarian

In [47]:
#imp faiss
#imp numpy
# pandas
#sentence transofrmer

model = SentenceTransformer('all-MiniLM-L6-v2')
index= faiss.read_index('knowledge_base.index')

knowledge_base_csv_df =pd.read_csv('knowledge_base_100k_csv_sample.csv')

#build our librarian function
def find_relevant_context(user_question,top_k=5):
    #Convert the new question into a vector
    question_embedding =model.encode([user_question])
    #Search the index for the closest matches      It then searches through our massive "map" (index) and instantly finds the 5 vectors that are mathematically closest to our new question's vector.
    # The search returns the distances and the indices (row numbers) of the top matches. it will use calcuate the use of the  embedding ones,
    distances,indices =index.search(question_embedding,top_k)
    #Retrieve the actual text using the results
    relevent_context =knowledge_base_csv_df.iloc[indices[0]]
    return relevent_context

#test the librarian
test_question = "My Amazon order is delayed, what should I do?"
similar_conservations =find_relevant_context(test_question,top_k=5)
print(f"\n for test question:'{test_question}'")
print("\n top 5 most relevant conservatins")
display(similar_conservations)


 for test question:'My Amazon order is delayed, what should I do?'

 top 5 most relevant conservatins


C:\Users\balap\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


,question,answer,cleaned_question
75853,Amazon just told me my package was delayed... ...,@793985 I am so sorry for the delay! Please le...,amazon just told me my package was delayed...
2484,My parcel is delayed till 12 Dec😭 Amazon...why?!,@274548 I'm really sorry to hear about that. W...,my parcel is delayed till 12 dec😭 amazon...why?!
35792,@115851 @115850 @118702 Amazon did not deliver...,"@322942 I get your disappointment, Pratik. Hav...",amazon did not deliver my order and cancelled ...
93975,@AmazonHelp Have been waiting for past 3 days....,"@298990 As you've shared your details, we'll w...",have been waiting for past 3 days..have sent c...
25200,"@115850 my order is quite delayed, what's the ...",@335267 Please share your details here: https:...,"my order is quite delayed, what's the recourse..."


# Expert Consultant

In [59]:
import google.generativeai as genai
#Configure the Gemini API client
genai.configure(api_key=gemini_api_key)

gemini_model =genai.GenerativeModel('models/gemini-2.5-flash')

def generate_smart_answer(user_question,context_df):
    '''
    this function takes user question and context form our search,
    and uses the gemini api to generate the smart, helpful answer
    '''
    # First, we format the context from the DataFrame into a simple text block.
    context_text = ""
    for index,row in context_df.iterrows():
        context_text += f"Historical question:{row['question']}\n official answer:{row['answer']}\n\n"

    #the special promt for the gemini model
    prompt = f"""
    You are an expert customer support agent. Your primary goal is to answer the user's question in a helpful and concise way, based *only* on the historical context provided.

    **Do not make up information.** If the context does not contain the answer, say that you don't have enough information to help.

    Here is the historical context:
    ---
    {context_text}
    ---

    Here is the user's new question:
    "{user_question}"

    Please provide your answer now.
    """
    # Make the API call to get the generated answer
    response=gemini_model.generate_content(prompt)
    return response.text

#test or final system
test_question = "My Amazon order is delayed, what should I do?"

# 1. First, our Librarian finds the context.
similar_conversations = find_relevant_context(test_question, top_k=5)

# 2. Then, our Consultant uses that context to generate the final answer.
final_answer = generate_smart_answer(test_question, similar_conversations)

print(f'user question:{test_question}')
print("\n final generated answer")
print("\n------------------")
print(final_answer)
    

C:\Users\balap\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
C:\Users\balap\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


user question:My Amazon order is delayed, what should I do?

 final generated answer

------------------
Please share your details via the provided link so we can get in touch with you: https://t.co/GIJyeYqKE0. Alternatively, if a new delivery date was given, please let us know if you don't receive it by that date.


# Secretary

In [69]:
#imp os
import requests

#Load our Airtable secrets
airtable_api_key =os.getenv("AIRTABLE_API_KEY")
airtable_base_id =os.getenv("AIRTABLE_BASE_ID")
airtable_table_name = 'Table 1'

# -- Build our "Secretary" function--
def create_support_ticket(question_text):
    '''
    this function takes a user question and create the new record
    in airtable "support ticket" base
    '''
    url = f"https://api.airtable.com/v0/{airtable_base_id}/{airtable_table_name}"
    headers ={
        "Authorization": f"Bearer {airtable_api_key}",
        "Content-Type": "application/json"
    }
    data ={
        'records':[
            {
                'fields':{
                    'User Question':question_text,
                    'Status':"Todo"
                }
            }
        ]
    }
    response =requests.post(url,headers=headers,json=data)
    if response.status_code ==200:
        print("success a new  support ticket was created in aritable.")
        return True
    else:
        print(f"errror creating ticket:{response.status_code}")
        print(response.text)
        return False

# --- let's test it! ---
test_ticket_question = "My account is locked and I cannot reset my password. I need to speak to a human."
create_support_ticket(test_ticket_question)         

success a new  support ticket was created in aritable.


True